In [0]:
CREATE SCHEMA IF NOT EXISTS delta;

Uso de apóstrofe no nome do schema.\
Se não especifica o schema ele é colocado como "default".

In [0]:
CREATE TABLE `delta`.deltaFile
(
    Education_Level VARCHAR(50),
    Line_Number INT,
    Employed INT,
    Unemployed INT,
    Industry VARCHAR(50),
    Gender VARCHAR(10),
    Date_Inserted DATE,
    dense_rank INT
)

A presença do diretório "`_delta_log`" indica que trata-se de um delta lake ou um delta table.\
Como não adicionamos nenhum dado ainda não temos parquet files 

In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1727744319000)]

In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2847, modificationTime=1727744323000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1778, modificationTime=1727744320000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/__tmp_path_dir/', name='__tmp_path_dir/', size=0, modificationTime=1727744319000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/_commits/', name='_commits/', size=0, modificationTime=1727744319000)]

Examinando o arquivo json dentro de _delta_log, temos informações sobre a operação "CREATE TABLE" e sobre o schema criado.

In [0]:
select *
from TEXT.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json`

value
"{""commitInfo"":{""timestamp"":1727744319746,""userId"":""2239208310904017"",""userName"":""lepina73@hotmail.com"",""operation"":""CREATE TABLE"",""operationParameters"":{""partitionBy"":""[]"",""clusterBy"":""[]"",""description"":null,""isManaged"":""true"",""properties"":""{\""delta.enableDeletionVectors\"":\""true\""}"",""statsOnLoad"":false},""notebook"":{""notebookId"":""4156582661638974""},""clusterId"":""1001-004428-4ucqgjdm"",""isolationLevel"":""WriteSerializable"",""isBlindAppend"":true,""operationMetrics"":{},""tags"":{""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/15.4.x-scala2.12"",""txnId"":""36e071bc-0cf4-482c-a71c-cd9d6c273a88""}}"
"{""metaData"":{""id"":""9d717c9a-09fb-4ead-ad91-a67f1d7886ee"",""format"":{""provider"":""parquet"",""options"":{}},""schemaString"":""{\""type\"":\""struct\"",\""fields\"":[{\""name\"":\""Education_Level\"",\""type\"":\""string\"",\""nullable\"":true,\""metadata\"":{\""__CHAR_VARCHAR_TYPE_STRING\"":\""varchar(50)\""}},{\""name\"":\""Line_Number\"",\""type\"":\""integer\"",\""nullable\"":true,\""metadata\"":{}},{\""name\"":\""Employed\"",\""type\"":\""integer\"",\""nullable\"":true,\""metadata\"":{}},{\""name\"":\""Unemployed\"",\""type\"":\""integer\"",\""nullable\"":true,\""metadata\"":{}},{\""name\"":\""Industry\"",\""type\"":\""string\"",\""nullable\"":true,\""metadata\"":{\""__CHAR_VARCHAR_TYPE_STRING\"":\""varchar(50)\""}},{\""name\"":\""Gender\"",\""type\"":\""string\"",\""nullable\"":true,\""metadata\"":{\""__CHAR_VARCHAR_TYPE_STRING\"":\""varchar(10)\""}},{\""name\"":\""Date_Inserted\"",\""type\"":\""date\"",\""nullable\"":true,\""metadata\"":{}},{\""name\"":\""dense_rank\"",\""type\"":\""integer\"",\""nullable\"":true,\""metadata\"":{}}]}"",""partitionColumns"":[],""configuration"":{""delta.enableDeletionVectors"":""true""},""createdTime"":1727744319649}}"
"{""protocol"":{""minReaderVersion"":3,""minWriterVersion"":7,""readerFeatures"":[""deletionVectors""],""writerFeatures"":[""deletionVectors""]}}"



## Adding records to delta table

Necessário especificar o schema para não ser criado dentro de `"default"`

In [0]:
INSERT INTO `delta`.deltafile
VALUES
    ('Bachelor', 1, 4500, 500, 'IT', 'Male', '2023-07-12',  1),
    ('Master', 2, 6500, 500, 'Finance', 'Female', '2023-07-12', 2),
    ('High School', 3, 3500, 500, 'Retail', 'Male', '2023-07-12', 3),
    ('PhD', 4, 5500, 500, 'Healthcare', 'Female', '2023-07-12', 4);

num_affected_rows,num_inserted_rows
4,4



## Querying the table

In [0]:
SELECT *
FROM `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
Bachelor,1,4500,500,IT,Male,2023-07-12,1
Master,2,6500,500,Finance,Female,2023-07-12,2
High School,3,3500,500,Retail,Male,2023-07-12,3
PhD,4,5500,500,Healthcare,Female,2023-07-12,4


Houve a geração de um parquet file com as informações dos dados coletados.

In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1727744319000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', name='part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', size=2556, modificationTime=1727744601000)]

Houve a geração de um arquivo json

In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2847, modificationTime=1727744323000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1778, modificationTime=1727744320000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.crc', name='00000000000000000001.crc', size=3773, modificationTime=1727744606000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json', name='00000000000000000001.json', size=1525, modificationTime=1727744602000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/__tmp_path_dir/', name='__tmp_path_dir/', size=0, modificationTime=1727744319000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/_commits/', name='_commits/', size=0, modificationTime=1727744319

In [0]:
select *
from TEXT.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json`

value
"{""commitInfo"":{""timestamp"":1727744602593,""userId"":""2239208310904017"",""userName"":""lepina73@hotmail.com"",""operation"":""WRITE"",""operationParameters"":{""mode"":""Append"",""statsOnLoad"":false,""partitionBy"":""[]""},""notebook"":{""notebookId"":""4156582661638974""},""clusterId"":""1001-004428-4ucqgjdm"",""readVersion"":0,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":true,""operationMetrics"":{""numFiles"":""1"",""numOutputRows"":""4"",""numOutputBytes"":""2556""},""tags"":{""noRowsCopied"":""true"",""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/15.4.x-scala2.12"",""txnId"":""4d5083e5-0243-496c-b7d8-dd56d8195947""}}"
"{""add"":{""path"":""part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet"",""partitionValues"":{},""size"":2556,""modificationTime"":1727744601000,""dataChange"":true,""stats"":""{\""numRecords\"":4,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":6500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0},\""tightBounds\"":true}"",""tags"":{""INSERTION_TIME"":""1727744601000000"",""MIN_INSERTION_TIME"":""1727744601000000"",""MAX_INSERTION_TIME"":""1727744601000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"



## Updating data in table

In [0]:
UPDATE `delta`.deltafile
SET Industry = 'Finance'
WHERE Education_Level = 'PhD'

num_affected_rows
1


In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2847, modificationTime=1727744323000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1778, modificationTime=1727744320000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.crc', name='00000000000000000001.crc', size=3773, modificationTime=1727744606000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json', name='00000000000000000001.json', size=1525, modificationTime=1727744602000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.crc', name='00000000000000000002.crc', size=4814, modificationTime=1727745006000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.json', name='0000000000

In [0]:
select *
from TEXT.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.json`

value
"{""commitInfo"":{""timestamp"":1727745002926,""userId"":""2239208310904017"",""userName"":""lepina73@hotmail.com"",""operation"":""UPDATE"",""operationParameters"":{""predicate"":""[\""(Education_Level#2763 = PhD)\""]""},""notebook"":{""notebookId"":""4156582661638974""},""clusterId"":""1001-004428-4ucqgjdm"",""readVersion"":1,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":false,""operationMetrics"":{""numRemovedFiles"":""0"",""numRemovedBytes"":""0"",""numCopiedRows"":""0"",""numDeletionVectorsAdded"":""1"",""numDeletionVectorsRemoved"":""0"",""numAddedChangeFiles"":""0"",""executionTimeMs"":""6614"",""numDeletionVectorsUpdated"":""0"",""scanTimeMs"":""2683"",""numAddedFiles"":""1"",""numUpdatedRows"":""1"",""numAddedBytes"":""2375"",""rewriteTimeMs"":""3854""},""tags"":{""noRowsCopied"":""true"",""delta.rowTracking.preserved"":""false"",""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/15.4.x-scala2.12"",""txnId"":""b426d905-18a4-4e7b-a9bb-129c54720c88""}}"
"{""remove"":{""path"":""part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet"",""deletionTimestamp"":1727745002892,""dataChange"":true,""extendedFileMetadata"":true,""partitionValues"":{},""size"":2556,""tags"":{""INSERTION_TIME"":""1727744601000000"",""MIN_INSERTION_TIME"":""1727744601000000"",""MAX_INSERTION_TIME"":""1727744601000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""},""stats"":""{\""numRecords\"":4,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":6500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0},\""tightBounds\"":false}""}}"
"{""add"":{""path"":""part-00000-192fda29-0f44-40b3-b159-73a9c5512207-c000.snappy.parquet"",""partitionValues"":{},""size"":2375,""modificationTime"":1727745000000,""dataChange"":true,""stats"":""{\""numRecords\"":1,\""minValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":5500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":5500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0},\""tightBounds\"":true}"",""tags"":{""MAX_INSERTION_TIME"":""1727744601000000"",""INSERTION_TIME"":""1727744601000000"",""MIN_INSERTION_TIME"":""1727744601000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"
"{""add"":{""path"":""part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet"",""partitionValues"":{},""size"":2556,""modificationTime"":1727744601000,""dataChange"":true,""stats"":""{\""numRecords\"":4,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":6500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0},\""tightBounds\"":false}"",""tags"":{""I


## Querying table after UPDATE

In [0]:
SELECT *
FROM `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
PhD,4,5500,500,Finance,Female,2023-07-12,4
Bachelor,1,4500,500,IT,Male,2023-07-12,1
Master,2,6500,500,Finance,Female,2023-07-12,2
High School,3,3500,500,Retail,Male,2023-07-12,3


In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1727744319000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/deletion_vector_c2db8c3a-546f-4256-92ed-9432bf715c07.bin', name='deletion_vector_c2db8c3a-546f-4256-92ed-9432bf715c07.bin', size=43, modificationTime=1727744998000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', name='part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', size=2556, modificationTime=1727744601000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-192fda29-0f44-40b3-b159-73a9c5512207-c000.snappy.parquet', name='part-00000-192fda29-0f44-40b3-b159-73a9c5512207-c000.snappy.parquet', size=2375, modificationTime=1727745000000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-23eaa0f8-cf89-44b6-bf4d-7b405455e303-c000.snappy.parquet', name


### Delet operation

In [0]:
DELETE FROM `delta`.deltafile
WHERE dense_rank =2

num_affected_rows
1


In [0]:
SELECT *
FROM `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
PhD,4,5500,500,Finance,Female,2023-07-12,4
Bachelor,1,4500,500,IT,Male,2023-07-12,1
High School,3,3500,500,Retail,Male,2023-07-12,3


In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1727744319000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/deletion_vector_7ba7cf31-0a1a-407b-91e3-88240f3ea44c.bin', name='deletion_vector_7ba7cf31-0a1a-407b-91e3-88240f3ea44c.bin', size=43, modificationTime=1727745396000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/deletion_vector_c2db8c3a-546f-4256-92ed-9432bf715c07.bin', name='deletion_vector_c2db8c3a-546f-4256-92ed-9432bf715c07.bin', size=43, modificationTime=1727744998000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', name='part-00000-058db2f9-644c-43a9-98d5-51ca1de2eb8c-c000.snappy.parquet', size=2556, modificationTime=1727744601000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-192fda29-0f44-40b3-b159-73a9c5512207-c000.snappy.parquet', name='part-00000-192fda29-0f

In [0]:
%python
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

In [0]:
select *
from TEXT.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000003.json`